In [1]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [2]:
df_train = pd.read_csv('train_with_features.csv')

In [3]:
df_train

,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,...,trade_log_return_realized_volatility_450_max_time,trade_log_return_realized_volatility_450_min_time,trade_log_return_realized_volatility_300_mean_time,trade_log_return_realized_volatility_300_std_time,trade_log_return_realized_volatility_300_max_time,trade_log_return_realized_volatility_300_min_time,trade_log_return_realized_volatility_150_mean_time,trade_log_return_realized_volatility_150_std_time,trade_log_return_realized_volatility_150_max_time,trade_log_return_realized_volatility_150_min_time
0,0,5,0.004136,0-5,303.125061,1.003725,0.000693,303.105539,1.003661,0.000781,...,0.003242,0.000543,0.001820,0.000692,0.004595,0.000710,0.002286,0.000836,0.005362,0.000888
1,0,11,0.001445,0-11,200.047768,1.000239,0.000262,200.041171,1.000206,0.000272,...,0.002701,0.000000,0.000906,0.000460,0.002783,0.000000,0.001140,0.000583,0.002851,0.000000
2,0,16,0.002168,0-16,187.913849,0.999542,0.000864,187.939824,0.999680,0.000862,...,0.002751,0.000114,0.001100,0.000428,0.003082,0.000497,0.001347,0.000484,0.003414,0.000717
3,0,31,0.002195,0-31,119.859781,0.998832,0.000757,119.835941,0.998633,0.000656,...,0.003404,0.000000,0.001052,0.000600,0.004218,0.000000,0.001349,0.000698,0.004974,0.000269
4,0,62,0.001747,0-62,175.932865,0.999619,0.000258,175.934256,0.999626,0.000317,...,0.001936,0.000158,0.000812,0.000372,0.002470,0.000278,0.001036,0.000466,0.003281,0.000317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,126,32751,0.003461,126-32751,309.870466,0.999582,0.000486,309.871372,0.999585,0.000613,...,0.003079,0.000000,0.001261,0.000485,0.004049,0.000534,0.001576,0.000651,0.004783,0.000647
428928,126,32753,0.003113,126-32753,223.552143,1.002476,0.001264,223.580314,1.002602,0.001303,...,0.003531,0.000000,0.001008,0.000687,0.006310,0.000329,0.001241,0.000814,0.007915,0.000404
428929,126,32758,0.004070,126-32758,256.277050,1.001082,0.000466,256.255056,1.000996,0.000599,...,0.001669,0.000000,0.001055,0.000376,0.001995,0.000000,0.001306,0.000422,0.002566,0.000000
428930,126,32763,0.003357,126-32763,399.721736,1.001809,0.000456,399.714332,1.001790,0.000507,...,0.003270,0.000400,0.001474,0.000591,0.005284,0.000686,0.001839,0.000731,0.006914,0.001004


In [4]:
kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
trn_idx, val_idx = first(kfold.split(df_train))

In [5]:
df_train.stock_id = df_train.stock_id.astype('category')

In [6]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [7]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [20]:
def train_fold(trn_idx, val_idx):
    

    procs_nn = [Categorify, FillMissing, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')

    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,500,200],
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe),
                        cbs=[EarlyStoppingCallback(monitor='rmspe', comp=np.less, patience=20)])
    learn.fit_one_cycle(100, 5e-3)
    return learn.early_stopping.best

In [13]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [14]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [19]:
# TD: nans

In [21]:
res =[]
for trn_idx, val_idx in kfold.split(df_train):
    res.append(train_fold(trn_idx, val_idx))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,2.788222,2.095870,2.180558,00:06
1,0.776841,0.517598,0.554577,00:06
2,0.464974,0.629344,0.767302,00:06
3,0.287846,0.254162,0.259588,00:06
4,0.265678,0.251952,0.264280,00:06
5,0.250205,0.245790,0.249736,00:06
6,0.242795,0.241395,0.244458,00:06
7,0.267763,0.260705,0.264429,00:06
8,0.239272,0.287052,0.410705,00:06
9,0.250794,0.236477,0.241008,00:06


fold 1 0.1995101124048233


epoch,train_loss,valid_loss,rmspe,time
0,2.902217,2.127527,2.209362,00:06
1,0.862696,0.615518,0.644402,00:06
2,0.448971,0.338417,0.352866,00:06
3,0.282720,0.248178,0.253970,00:06
4,0.256847,0.272770,0.282372,00:06
5,0.251218,0.237554,0.241922,00:06
6,0.253664,0.273974,0.279735,00:06
7,0.244346,0.233942,0.236534,00:06
8,0.272932,0.230155,0.235754,00:06
9,0.249063,0.350238,1.153849,00:06


No improvement since epoch 70: early stopping
fold 2 0.19795624911785126


epoch,train_loss,valid_loss,rmspe,time
0,2.936600,2.135570,2.192801,00:06
1,0.811041,0.567089,0.604796,00:06
2,0.500652,0.337002,0.357709,00:06
3,0.343508,0.313797,0.315914,00:06
4,0.258457,0.268221,0.274685,00:06
5,0.265124,0.257657,0.263859,00:06
6,0.241957,0.231091,0.236791,00:06
7,0.235871,0.221905,0.224973,00:06
8,0.229980,0.223633,0.226571,00:06
9,0.262667,0.348499,0.352777,00:06


...

epoch,train_loss,valid_loss,rmspe,time
0,2.666707,2.005033,2.104029,00:06
1,0.847331,0.560591,0.604674,00:06
2,0.524667,0.382699,0.529154,00:06
3,0.310887,0.276214,0.309573,00:06
4,0.257237,0.241363,0.246463,00:06
5,0.250353,0.246468,0.254247,00:06
6,0.295401,0.244737,0.255721,00:06
7,0.245951,0.250015,0.252058,00:06
8,0.260099,0.229215,0.232945,00:06
9,0.238511,0.227874,0.232139,00:06


No improvement since epoch 69: early stopping
fold 4 0.20451131463050842


epoch,train_loss,valid_loss,rmspe,time
0,2.855625,2.055436,2.114556,00:06
1,0.894417,0.569011,0.616448,00:06
2,0.575838,0.787005,0.929362,00:06
3,0.306647,0.253261,0.259486,00:06
4,0.256632,0.238380,0.243170,00:06
5,0.260845,0.237412,0.243699,00:06
6,0.246816,0.234541,0.239075,00:06
7,0.261738,0.233000,0.237491,00:06
8,0.230427,0.226426,0.230062,00:06
9,0.232195,0.228939,0.232199,00:06


fold 5 0.19957980513572693


(0.19984760284423828,
 [0.1995101124048233,
  0.19795624911785126,
  0.1976805329322815,
  0.20451131463050842,
  0.19957980513572693])

In [42]:
learn.early_stopping.best

0.206015944480896

In [26]:
learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,500,200],
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe), opt_func=ranger)

In [27]:
learn.fit_flat_cos(100, 5e-3)

epoch,train_loss,valid_loss,rmspe,time
0,1.815724,0.922087,0.929451,00:04
1,0.527516,1.175475,1.195267,00:04
2,0.282976,0.248377,0.250462,00:04
3,0.245044,0.566122,0.582594,00:04
4,0.244354,0.233376,0.236646,00:04
5,0.231451,0.226809,0.228695,00:04
6,0.241084,0.226648,0.228408,00:04
7,0.230638,0.242259,0.244876,00:04
8,0.228333,0.222964,0.224587,00:04
9,0.231134,0.232729,0.234192,00:04


In [100]:
preds, tars = learn.get_preds()

In [101]:
preds.shape

torch.Size([85787, 1])

In [102]:
preds_nn = preds.view(-1).numpy()

In [103]:
rmspe_np(y_true = y_valid, y_pred = preds_nn)

0.22675025973279012

In [104]:
preds_ens = (y_pred + preds_nn)/2

In [105]:
rmspe_np(y_true = y_valid, y_pred = preds_ens)

0.22713026508344736

In [106]:
learn.save('tabular226.pth')

Path('models/tabular226.pth.pth')